![MicrosoftFilm](./images/MicrosoftFilm.jpg)

# Microsoft Studios: Film Production Strategy

**Authors:** Jacob Heyman, Mitch Krieger
***
## Overview

Using data from IMDB, we investigated trends in films the past 10 years in order to best reccomend to Microsoft what strategies to employ to launch their new video and content creation division. We analyized various metrics of a film's success including, gross revenues, net profit and return on investment (percentage gross) against a variety of factors such as genre, rating, runtime, and production budget. Our findings indicate that Microsoft should focus on producing new films with conservative budgets with an emphasis on certain genres. In addition, Microsoft should not be concerned with high-profile principle cast and crew, nor should they be overly concerned with runtime, performance internationally or movie-goer reviews as our data suggests that these factors are not correlated with a film's financial success. 

## Business Problem

In order to stay competitive with existing tech companies like Apple and Amazon, Microsoft wishes to start their own video content creation studio. To best launch their new venture, Microsoft hopes to look at trends in movie data in order to focus on producing the most competitive content. From these trends, Microsoft is seeking actionable insights to spearhead the opening of their movie studio and produce data-driven projects. A challenege for Microsoft will be differentiating themselves in the crowded field of content creation, and determining what factors most affect a film's success.

**Research Questions to Consider:**
To provide the best insights for Microsoft's new studio, we focused on movie data from the past 10 years using profit and percentage gross as metrics of success and what factors affect that success. With this in mind we developed the following question to investigate:

1. Which genre of movies has the highest average net **profit** over the past 10 years?
2. Which genre of movies has the greatest percentage return on investment (percentage gross)?
3. What is the relationsship between box office gross/ROIT (percentage gross) and the following factors?
   - IMDB Rating
   - Runtime
   - Budget
4. Does director/writer/acting choices affect the profitiability of a movie? 
5. What is the relationship between domestic gross and foreign gross?

## Data Understanding
To answer the above questions, we collected various datasets from IMDB and the Box Office. Then, we created a new set of data to analyze the relevant factors that affect our metrics of success and our research questions. 

In [1]:
import pandas as pd
import numpy as np
from csv_to_clean_dataframes import *

### Data Sets:
The following are the sources of data we used:
* imdb_title_df - **IMDB Movie titles & genre** 
* imdb_name_df - **IMDB Staff Names & Jobs** 
* imdb_title_crew_df - **IMDB Movie Directors & Writers** 
* imdb_principles_df - **IMDB Movie Principle Roles & Crew** 
* imdb_title_rating_df - **IMDB Ratings** 
* movie_budgets_df - **Box Office Grosses & Budgets** 

Each movie and crew/cast member comes with a `tconst` and `nconst` unique identifier respectively. Using these unique identifiers, we organized the tables into the following schema so that we could join datasets between the IMDB datasets, then for information from the box office we joined on title. Original csv files can be found in the `/zippedData` folder.

### Schema:
![DatabaseSchema](images/Movie-Tables-Schema.png)


## Data Preparation

### Data cleaning
From these datasets we eliminated any irrelevant data and dropped entries with missing values for production budget. And did a 1:1 left join of imdb_title_df and box office gross using `primary_name` imdb_title rating using `tconst` to create the tables below.

### Feature Engineering
We also added a `worldwide_net` feature by subtracting `worldwide_gross` from `production_budget` and a `percent_gross` feature by subtracting `worldwide_gross` by `production_budget`. These features help to better understand return on investment. Code for cleaning and feature engineering can be found in `csv_to_clean_dataframes.py`.

In [2]:
joined_table.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,production_budget,domestic_gross,worldwide_gross,worldwide_net,foreign_gross,percent_gross,averagerating
20,tt0249516,Foodfight!,Foodfight!,2012,91.0,"Action,Animation,Comedy",45000000.0,0.0,73706.0,-44926294.0,73706.0,-99.836209,1.9
33,tt0293429,Mortal Kombat,Mortal Kombat,2021,NaN,"Action,Adventure,Fantasy",20000000.0,70433227.0,122133227.0,102133227.0,51700000.0,510.666135,NaN
40,tt0326592,The Overnight,The Overnight,2010,88.0,NaN,200000.0,1109808.0,1165996.0,965996.0,56188.0,482.998000,7.5
48,tt0337692,On the Road,On the Road,2012,124.0,"Adventure,Drama,Romance",25000000.0,720828.0,9313302.0,-15686698.0,8592474.0,-62.746792,6.1
54,tt0359950,The Secret Life of Walter Mitty,The Secret Life of Walter Mitty,2013,114.0,"Adventure,Comedy,Drama",91000000.0,58236838.0,187861183.0,96861183.0,129624345.0,106.440860,7.3


We kept missing values for runtime, but created multiple dataframes to keep as many entries as possible depending upon the needs of the analysis we were performing:

In [9]:
#No missing values for gross:
noZero_gross.isna().sum()

tconst                 0
primary_title          0
original_title         1
start_year             0
runtime_minutes      408
genres                63
production_budget      0
domestic_gross         0
worldwide_gross        0
worldwide_net          0
foreign_gross          0
percent_gross          0
averagerating        808
dtype: int64

In [4]:
#one to one join to incoroporate writers & directors
crew_grosses_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,production_budget,domestic_gross,worldwide_gross,worldwide_net,foreign_gross,percent_gross,averagerating,directors,writers
0,tt0293429,Mortal Kombat,Mortal Kombat,2021,NaN,"Action,Adventure,Fantasy",20000000.0,70433227.0,122133227.0,102133227.0,51700000.0,510.666135,NaN,nm2585406,nm3859643
1,tt0326592,The Overnight,The Overnight,2010,88.0,NaN,200000.0,1109808.0,1165996.0,965996.0,56188.0,482.998000,7.5,nm1208371,"nm1170333,nm1208371"
2,tt0337692,On the Road,On the Road,2012,124.0,"Adventure,Drama,Romance",25000000.0,720828.0,9313302.0,-15686698.0,8592474.0,-62.746792,6.1,nm0758574,"nm0449616,nm1433580"
3,tt0359950,The Secret Life of Walter Mitty,The Secret Life of Walter Mitty,2013,114.0,"Adventure,Comedy,Drama",91000000.0,58236838.0,187861183.0,96861183.0,129624345.0,106.440860,7.3,nm0001774,"nm0175726,nm0862122"
4,tt0365907,A Walk Among the Tombstones,A Walk Among the Tombstones,2014,114.0,"Action,Crime,Drama",28000000.0,26017685.0,62108587.0,34108587.0,36090902.0,121.816382,6.5,nm0291082,"nm0088747,nm0291082"


In [5]:
#one to many join to incorporate principle roles (actors,writers,directors, composers, and so on)
principles_grosses_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,production_budget,domestic_gross,worldwide_gross,worldwide_net,foreign_gross,percent_gross,averagerating,ordering,nconst,category,job,characters
0,tt0293429,Mortal Kombat,Mortal Kombat,2021,NaN,"Action,Adventure,Fantasy",20000000.0,70433227.0,122133227.0,102133227.0,51700000.0,510.666135,NaN,1,nm2585406,director,NaN,NaN
1,tt0293429,Mortal Kombat,Mortal Kombat,2021,NaN,"Action,Adventure,Fantasy",20000000.0,70433227.0,122133227.0,102133227.0,51700000.0,510.666135,NaN,2,nm3859643,writer,screenplay,NaN
2,tt0293429,Mortal Kombat,Mortal Kombat,2021,NaN,"Action,Adventure,Fantasy",20000000.0,70433227.0,122133227.0,102133227.0,51700000.0,510.666135,NaN,3,nm0307776,producer,producer,NaN
3,tt0293429,Mortal Kombat,Mortal Kombat,2021,NaN,"Action,Adventure,Fantasy",20000000.0,70433227.0,122133227.0,102133227.0,51700000.0,510.666135,NaN,4,nm0440415,producer,producer,NaN
4,tt0293429,Mortal Kombat,Mortal Kombat,2021,NaN,"Action,Adventure,Fantasy",20000000.0,70433227.0,122133227.0,102133227.0,51700000.0,510.666135,NaN,5,nm0003342,producer,producer,NaN


In [10]:
#no missing values for genres:
droped_genres_dataframe.isna().sum()

tconst                 0
primary_title          0
original_title         0
start_year             0
runtime_minutes      361
genres                 0
production_budget      0
domestic_gross         0
worldwide_gross        0
worldwide_net          0
foreign_gross          0
percent_gross          0
averagerating        753
dtype: int64

## Analysis
We then decided to focus how on various factors (year, runtime_minutes, genre, production_budget, principle_crew (nmconst), averagerating) effect the gross profit and percent gross metrics in our analysis. [Analysis can be found in this notebook](https://github.com/Jacobheyman702/Movie-data-analysis/blob/master/analysis.ipynb).